In [6]:
import ujson
import glob
import os
import re
import pandas as pd
from nltk.parse.corenlp import CoreNLPDependencyParser, CoreNLPParser
parser = CoreNLPDependencyParser(url='http://localhost:9010')

In [7]:
# load the non-dreprecated java doc caveat sentences of methods (parameters or exception level)
caveat_files_dir = './output/java_12_spec_caveat_sentences_revised/'
parameter_caveats = []
exception_caveats = []

files = sorted(glob.glob(caveat_files_dir + '*.json'))
for file in files:
    with open(file) as f:
        arr = ujson.load(f)
        full_class_name = os.path.splitext(os.path.basename(file))[0]
        simple_class_name = full_class_name.split('.')[-1]
        
        for caveat in arr:
            if not caveat['deprecated'] and 'name' in caveat:
                
                # collect the name of parameters
                collected = False
                parameters = []
                for misc_obj in caveat['caveat_misc']:
                    if misc_obj['name'] == 'Parameters:':
                        for obj in misc_obj['list']:
                            parameters.append(obj['parameter'])
                        collected = True
                        break

                if collected:
                    # add all parameter and exception level sentences
                    for misc_obj in caveat['caveat_misc']:
                        if misc_obj['name'] in ['Parameters:', 'Throws:']:
                            for obj in misc_obj['list']:
                                for misc_sentence in obj['sentences']:
                                    if misc_obj['name'] == 'Parameters:':
                                        e = {
                                            'obj': obj['parameter'],
                                            'simple_class_name': simple_class_name,
                                            'full_class_name': full_class_name,
                                            'api': caveat['name'],
                                            'signature': caveat['signature'],
                                            'sentence': misc_sentence,
                                            'parameters': parameters,
                                            'type': 'parameter'
                                        }
                                        parameter_caveats.append(e)
                                    else:
                                        e = {
                                            'obj': obj['exception'],
                                            'simple_class_name': simple_class_name,
                                            'full_class_name': full_class_name,
                                            'api': caveat['name'],
                                            'signature': caveat['signature'],
                                            'sentence': misc_sentence,
                                            'parameters': parameters,
                                            'type': 'exception'
                                        }
                                        exception_caveats.append(e)

parameter_caveat_df = pd.DataFrame(parameter_caveats)
exception_caveat_df = pd.DataFrame(exception_caveats)

In [ ]:
def extract_not_null_rules:
    nouns = []
    

In [5]:
parse, = parser.raw_parse("if the supplied class or array is null.")
for governor, dep, dependent in parse.triples():
    print(governor, dep, dependent)  # doctest: +NORMALIZE_WHITESPACE

('null', 'JJ') mark ('if', 'IN')
('null', 'JJ') nsubj ('class', 'NN')
('class', 'NN') det ('the', 'DT')
('class', 'NN') amod ('supplied', 'VBN')
('class', 'NN') cc ('or', 'CC')
('class', 'NN') conj ('array', 'NN')
('null', 'JJ') cop ('is', 'VBZ')
('null', 'JJ') punct ('.', '.')
